In [ ]:
def testgendata(mddf, andf):

#     imgHt   =   1080
#     imgWd   =   1920
#     imglist = []

    for index, row in mddf.iterrows():
        print('Processing file ({}): {}'.format(index + 1, row['video_name']))
        vidcap = cv2.VideoCapture(row['video_path'])

        frame_count = 0

        while(vidcap.isOpened()):
            res, frame = vidcap.read()
            if res == True:
                vidfile = os.path.join(tgtfld, row['video_name'].split('.')[0] + '_{}.jpg'.format(frame_count))
                cv2.imwrite(vidfile, frame)
                # print('Image File: {}'.format(vidfile))
                annotfile = os.path.join(tgtfld, row['video_name'].split('.')[0] + '_{}.txt'.format(frame_count))
                framedf = andf[(andf['video_name'] == row['video_name']) & (andf['frame_num'] == frame_count)].copy()
                # Convert left top coordinates to centre coordinates
                framedf['bb_lt_x'] = framedf['bb_lt_x'] + framedf['bb_width'] / 2
                framedf['bb_lt_y'] = framedf['bb_lt_y'] + framedf['bb_height'] / 2
                # Normalize the coordinates
                framedf['bb_lt_x']   =   framedf['bb_lt_x']   / imgWd
                framedf['bb_lt_y']   =   framedf['bb_lt_y']   / imgHt
                framedf['bb_width']  =   framedf['bb_width']  / imgWd
                framedf['bb_height'] =   framedf['bb_height'] / imgHt
                file = open(annotfile, mode='w+')
                # file.write(framedf.to_string(header=False, index=False))    # this option is generating extra spaces
                file.writelines('{} {} {} {} {}\n'.format(0, line[2], line[3], line[4], line[5]) for line in framedf.values)
                file.close()
                # print('Image File: {}'.format(annotfile))
                imglist.append(vidfile)

                frame_count += 1

                if cv2.waitKey(1) & 0xFF == ord('q'):
                    break

            else:
                break

        print('No. of images written for this video is {}'.format(frame_count))
        vidcap.release()

    imgpath = os.path.join(tgtfld + '/image_path.txt')
    file = open(imgpath, mode='w+')
    file.writelines('{}\n'.format(line) for line in imglist)
    file.close()
    print('Image Path is written in the file {}'.format(imgpath))